<a href="https://githubtocolab.com/gee-community/geemap/blob/master/docs/notebooks/118_download_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

**Download images directly to a local computer**

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install -U geemap

In [1]:
import os
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-eslamelnahas-jupyter')

In [2]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Download image by tile

In [4]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2020-12-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
  
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(31, 31, 8)
m.add_layer(dataset.mean(), visualization, 'RGB')
m

Map(center=[31, 31], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [5]:
out_dir = os.path.expanduser("~/Downloads")

In [6]:
region = ee.Geometry.BBox(30.45, 31.35, 30.55, 31.25)
m.centerObject(region)

In [7]:
features = geemap.fishnet(region,h_interval=0.05, v_interval=0.05,delta=1.0)
m.addLayer(features, {}, "Grids")

In [8]:
image = dataset.mean()

In [10]:
geemap.download_ee_image_tiles(
   image , features, out_dir="tiles", prefix="Sent_", crs="EPSG:4326", scale=5
)

Sent_1.tif: |                                                          | 0.00/129M (raw) [  0.0%] in 00:00 (et…

There is no STAC entry for: None


Sent_2.tif: |                                                          | 0.00/129M (raw) [  0.0%] in 00:00 (et…

Sent_3.tif: |                                                          | 0.00/129M (raw) [  0.0%] in 00:00 (et…

Sent_4.tif: |                                                          | 0.00/129M (raw) [  0.0%] in 00:00 (et…

...

Sent_6.tif: |                                                          | 0.00/129M (raw) [  0.0%] in 00:00 (et…

Sent_7.tif: |                                                          | 0.00/129M (raw) [  0.0%] in 00:00 (et…

Sent_8.tif: |                                                          | 0.00/129M (raw) [  0.0%] in 00:00 (et…

Sent_9.tif: |                                                          | 0.00/129M (raw) [  0.0%] in 00:00 (et…

Downloaded 9 tiles in 409.8736867904663 seconds.


In [ ]:
Map.user_roi.getInfo()